In [6]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
import logging
import sys
import re
from tqdm import tqdm
from IPython.display import HTML
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchmetrics
from torch.utils.data import DataLoader, Subset
from torch.nn import functional as F
import clip
from PIL import Image
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import pathlib
import json
from datasets import _transform, get_full_class_labels, fix_class_names, idx_to_label, CUBDataset
openai.api_key  = os.getenv('OPENAI_API_KEY')

# Dataset

In [7]:
CUB_DIR = '/Users/kushagrashekhawat/Desktop/CUB_200_2011/CUB_200_2011' # REPLACE THIS WITH YOUR OWN PATH
# load CUB dataset
data_dir = pathlib.Path(CUB_DIR)
cub_dataset = CUBDataset(data_dir, train=True, transform=_transform(224))

# Load model

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device, jit=False)
model = model.eval()
model = model.requires_grad_(False)

# Function for computing Label text emebddings

In [9]:
def compute_encodings(model, labels):
    label_encodings = F.normalize(model.encode_text(clip.tokenize(labels).to(device)))
    return label_encodings

# Function for index to label

In [10]:
def idx_to_label(label_to_idx):
    dataset_items = list(label_to_idx.items())
    return dict(map(lambda x: x[::-1], dataset_items))

In [11]:
def initialize_counter_dict(labels):
    dict_ = OrderedDict()
    for label1 in labels:
        dict_[label1] = OrderedDict()
    return dict_

In [12]:
def increase_count(counter_dict, actual_class, top_classes):
    for class_ in top_classes:
        if class_ in counter_dict[actual_class]:
            counter_dict[actual_class][class_] += 1
        else:
            counter_dict[actual_class][class_] = 1
    return counter_dict

In [13]:
def add_to_counter_dict(counter_dict, index_to_class_dict, actual_index_list, 
                        pred_index_list):
    for idx, actual_class_index in enumerate(actual_index_list):
        actual_class = index_to_class_dict[int(actual_class_index)]
        top_classes = list(map(lambda x: index_to_class_dict[int(x)], 
                               pred_index_list[idx]))
        counter_dict = increase_count(counter_dict, actual_class, top_classes)
    return counter_dict

# Calculate CLIP accuracy

In [15]:
def calc_acc(model, dataset, descriptions, batch_size = 1, count_classes = False, 
             topK = 5):
    
    
    org_topK = topK
    topK = max(org_topK, 5)
    if topK == 1:
        topK = 5
        topK_is_one = True
    elif topK < 1:
        return "Please use k >= 1!"
    
    print(org_topK)
    
    count_classes_dict = initialize_counter_dict(dataset.classes)
    index_to_class_dict = idx_to_label(dataset.class_to_idx)
    
    
    encodings = compute_encodings(model, descriptions)
    clip_accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(dataset.classes)).to(device)
    clip_accuracy_metric_topk = torchmetrics.Accuracy(top_k=topK, task="multiclass", num_classes=len(dataset.classes)).to(device)
    dataloader = DataLoader(dataset, batch_size, shuffle=True, num_workers=16, pin_memory=True)
    for batch_number, batch in enumerate(tqdm(dataloader)):
        images, labels = batch

        images = images.to(device)
        labels = labels.to(device)

        image_encodings = model.encode_image(images)
        image_encodings = F.normalize(image_encodings)

        image_labels_similarity = image_encodings @ encodings.T
        topk_values, topk_indices = image_labels_similarity.topk(topK, dim=1)
        clip_predictions = image_labels_similarity.argmax(dim=1)


        clip_acc = clip_accuracy_metric(image_labels_similarity, labels)
        clip_acc_topk = clip_accuracy_metric_topk(image_labels_similarity, labels)
        
        if count_classes == True:
            count_classes_dict = add_to_counter_dict(count_classes_dict, index_to_class_dict,
                                                 labels, topk_indices[:, 0:org_topK])


    accuracy_logs = {}
    accuracy_logs["Total CLIP-Standard Top-1 Accuracy: "] = 100*clip_accuracy_metric.compute().item()
    accuracy_logs[f"Total CLIP-Standard Top-{5} Accuracy: "] = 100*clip_accuracy_metric_topk.compute().item()
    
    if count_classes == True:
        for actual_key in count_classes_dict.keys():
            count_classes_dict[actual_key] = sorted(count_classes_dict[actual_key].items(),
                                                key=lambda x: x[1], reverse=True)

        json_string = json.dumps(count_classes_dict, indent=4)  # indent for pretty formatting

        # Write JSON string to a text file
        with open(f'top{org_topK}_cub.json', 'w') as file:
            file.write(json_string)

    
    return accuracy_logs

In [16]:
calc_acc(model, cub_dataset, cub_dataset.classes,
         count_classes = True, topK = 1)

5


  0%|                                                                                                                          | 0/5994 [00:00<?, ?it/s]


AttributeError: Can't pickle local object '_transform.<locals>.<lambda>'

# Graphs

In [14]:
def create_graph(actual_label, top5_pred):
    top5labels = list(map(lambda x: x[0], top5_pred))
    top5counts = list(map(lambda x: x[1], top5_pred))
    colors = ['royalblue' if label == actual_label else 'lightsteelblue' for label in top5labels]
    plt.barh(top5labels, top5counts, color=colors)

    num_ticks = 6  # Adjust the number of ticks as needed
    tick_positions = np.linspace(0, max(top5counts), num_ticks)
    plt.xticks(tick_positions, rotation=45, ha='right')  # Set the rotation and horizontal alignment

    plt.xlabel('Top-5 counts')
    plt.ylabel('Classes')
    plt.title('Score-wise Diagram')
    
    # Show the plot
    plt.show()


In [15]:
with open('top5_cub.json', 'r') as file:
    top1 = json.load(file)

for class_ in top1.keys():
    greater_than_zero = []
    for top_item in top1[class_]:
        if top_item[1] > 0:
            greater_than_zero.append(top_item)
    top1[class_] = greater_than_zero
#     create_graph(class_, top[class_][:min(len(greater_than_zero), 5)])

In [16]:
with open('top5_cub.json', 'r') as file:
    top5 = json.load(file)

for class_ in top1.keys():
    greater_than_zero = []
    for top_item in top5[class_]:
        if top_item[1] > 0:
            greater_than_zero.append(top_item)
    top5[class_] = greater_than_zero
#     create_graph(class_, top[class_][:min(len(greater_than_zero), 5)])

In [17]:
top1['Parakeet Auklet']

# [['Common Yellowthroat', 27],
#  ['Tennessee Warbler', 2],
#  ['Hooded Warbler', 1],
#  ['Black-footed Albatross', 0],
#  ['Laysan Albatross', 0]]

[['Rhinoceros Auklet', 29],
 ['Least Auklet', 29],
 ['Crested Auklet', 26],
 ['Parakeet Auklet', 25],
 ['Pigeon Guillemot', 15],
 ['Red-faced Cormorant', 6],
 ['Horned Puffin', 5],
 ['Red-legged Kittiwake', 3],
 ['Sooty Albatross', 1],
 ['Black-footed Albatross', 1],
 ['Gadwall', 1],
 ['Bank Swallow', 1],
 ['Pelagic Cormorant', 1],
 ['Brandt Cormorant', 1],
 ['Rock Wren', 1],
 ['Red-headed Woodpecker', 1],
 ['Dark-eyed Junco', 1],
 ['Black Tern', 1],
 ['Artic Tern', 1],
 ['Forsters Tern', 1]]

# Top-5 Using Labels and see a comparison with Top-1 using images

In [18]:
encodings = compute_encodings(model, cub_dataset.classes)
sim = encodings @ encodings.T
topk_values, topk_indices = sim.topk(20, dim = 1)
index_to_class_dict = idx_to_label(cub_dataset.class_to_idx)
counter_dict = initialize_counter_dict(cub_dataset.classes)
counter_dict = add_to_counter_dict(counter_dict, index_to_class_dict, topk_indices[:, 0], 
                        topk_indices)

# Top-1 and Top-5 Image overlap

In [19]:
overlap, total = 0, 0
for class_ in top1.keys():
    print("Actual Class: ", class_)
    image_class_set = set([image_class if class_ != image_class else None for image_class, value in top1[class_][0: 6]])
    total += len(image_class_set)
    counter = 0
    for label_class, val in top5[class_][0:5]:
        if label_class in image_class_set:
            print(label_class)
            overlap += 1
        if counter >= 6:
            break
    print("------"*10)
print(f"{overlap}/{total}")

# Overlap when using labels = 737/1035


Actual Class:  Black-footed Albatross
Sooty Albatross
Laysan Albatross
Northern Fulmar
Brown Pelican
------------------------------------------------------------
Actual Class:  Laysan Albatross
Sooty Albatross
Black-footed Albatross
Long-tailed Jaeger
Frigatebird
------------------------------------------------------------
Actual Class:  Sooty Albatross
Black-footed Albatross
Laysan Albatross
Long-tailed Jaeger
Rhinoceros Auklet
------------------------------------------------------------
Actual Class:  Groove-billed Ani
White-necked Raven
American Crow
Common Raven
Boat-tailed Grackle
------------------------------------------------------------
Actual Class:  Crested Auklet
Rhinoceros Auklet
Least Auklet
Parakeet Auklet
Pigeon Guillemot
------------------------------------------------------------
Actual Class:  Least Auklet
Parakeet Auklet
Crested Auklet
Rhinoceros Auklet
Pigeon Guillemot
------------------------------------------------------------
Actual Class:  Parakeet Auklet
Rhino

# Top-1 and Top-5 Label overlap

In [20]:
overlap, total = 0, 0
for class_ in top1.keys():
    print("Actual Class: ", class_)
    image_class_set = set([image_class if class_ != image_class else None for image_class, value in top1[class_][0: 6]])
    total += len(image_class_set)
    counter = 0
    for label_class in counter_dict[class_]:
        if label_class in image_class_set:
            print(label_class)
            overlap += 1
        if counter >= 5:
            break
    print("------"*10)
print(f"{overlap}/{total}")

# Overlap when using labels = 737/1035


Actual Class:  Black-footed Albatross
Sooty Albatross
Laysan Albatross
Northern Fulmar
Brown Pelican
------------------------------------------------------------
Actual Class:  Laysan Albatross
Sooty Albatross
Black-footed Albatross
Slaty-backed Gull
Frigatebird
Long-tailed Jaeger
------------------------------------------------------------
Actual Class:  Sooty Albatross
Black-footed Albatross
Laysan Albatross
Long-tailed Jaeger
------------------------------------------------------------
Actual Class:  Groove-billed Ani
Boat-tailed Grackle
American Crow
Common Raven
Bronzed Cowbird
White-necked Raven
------------------------------------------------------------
Actual Class:  Crested Auklet
Least Auklet
Rhinoceros Auklet
Parakeet Auklet
Pigeon Guillemot
Red-faced Cormorant
------------------------------------------------------------
Actual Class:  Least Auklet
Crested Auklet
Parakeet Auklet
Rhinoceros Auklet
Pigeon Guillemot
------------------------------------------------------------


# Generate attributes functions

In [22]:
def logs():
    logging.basicConfig(
        level=logging.DEBUG,            # Set the minimum log level to DEBUG
        filename='gpt_3_my_cub.log',        # Specify the log file name
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )

    logger = logging.getLogger(__name__)
    return logger


In [23]:
def get_completion(prompt, logger, model="gpt-3.5-turbo", temperature=0., max_tokens=300):
    messages = [{'role':'system', 'content':"""
    Use this as an example to generate you answers for user's query. Give the answer in a JSON format just like given in the example

    ```Q: What are useful visual features for distinguishing a tiger from a lion in a photo?
       A: There are several useful visual features to tell the difference between a tiger and a lion in a photo:-


        {
            "tiger": [
                     "has orange coats with vertical 'black' stripes",
                     "are generally larger and robust",
                     "ears are usually smaller and rounded",
                     "tails are long and have a characteristic 'white spot' at the tip",
                     "habitats are dense forests and grasslands"
                     ],

            "lion": [
                    "have a tawny or 'beige coat', and their fur is often smoother",
                    "has a more compact and muscular build",
                    "ears are large and often have a pointed shape",
                    "tails are shorter, ending in a tuft of hair",
                    "habitats are savannas and open woodlands"
                    ]
        }
        ```
    Remember not to use the first class name in the response of second class and vice-a-versa. For example here, in the answer for tiger do not use lion and in answer for lion do not use the word tiger.
"""},
    {"role": "user", "content": prompt}]
    
    try:
        logger.debug(f'Input: {messages}')
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        logger.debug(f'Output: {response}')
    except Exception as e:
        logger.error(f'error: {str(e)}')
        raise e
    return response.choices[0].message["content"]

# Create Attributes of Pairs from Top-1 Distribution

In [24]:
def generate_attributes(class1, class2):
    logger = logs()
    
    prompt = f"""
    Q: What are useful features for distinguishing a {class1} from {class2} in a photo?
    A: There are several useful visual features to tell the difference between a {class1} and a {class2} in a photo
    """
    print(prompt)
    response = get_completion(prompt, logger)
#     response = get_completion_dummy(prompt)
    answer = response
    print(response)
    
    return answer

In [25]:
def generate_attributes_dummy(class1, class2):
    logger = logs()
    
    prompt = f"""
    Q: What are useful features for distinguishing a {class1} from {class2} in a photo?
    A: There are several useful visual features to tell the difference between a {class1} and a {class2} in a photo
    """
#     print(prompt)
    return """aasdasd
    asfasfasfdas
    {
    "Laysan Albatross": [
    "have a white head and neck with a dark eye patch",
    "have a black upperwing with a white trailing edge",
    "underside is mostly white with dark edges on the underwing",
    "have a pink bill with a dark tip",
    "are generally medium-sized with a wingspan of around 6 feet",
    "nest in the Northwestern Hawaiian Islands"
    ],

    "Sooty Albatross": [
        "have a dark plumage overall, including the head and neck",
        "have a uniformly dark upperwing without a distinct pattern",
        "underside is dark, often with paler edges on the underwing",
        "have a dark bill without significant markings",
        "are larger in size with a wingspan of around 7 feet",
        "nest on sub-Antarctic islands and some isolated islands in the Southern Ocean"
    ]
    }sfsdfsdfsdf"""

In [26]:
def process_response(response):
    try:
        json_response = response
        json.loads(json_response)
    except Exception as e:
        lines = response.split('\n')
        json_response = ''
        for index, line in enumerate(lines):
            if line.find('{') != -1:
                json_response = line[line.find('{'):] + "\n"
                index += 1
                while index < len(lines) and lines[index].find('}') == -1:
                    json_response += lines[index]
                    index += 1
                if index < len(lines) and lines[index].find('}') != -1:
                    json_response += lines[index][:lines[index].find('}')+1]
                    break
                else:
                    html_string = f'<font color="red">{str(response)}</font>'
                    display(HTML(html_string))
    return json.loads(json_response)

In [27]:
def convert_to_json(diff_dict):
#     dict_ = diff_dict.copy()
#     for key in dict_.keys():
#         dict_[key] = list(dict_[key])
    return json.dumps(diff_dict, indent=4)

In [30]:
diff_dict = OrderedDict()
already_seen = set()
total_pairs = 0

In [31]:
import time
for class_ in tqdm(top1.keys()):
    if class_ not in diff_dict:
        diff_dict[class_] = []
    number_of_pairs = 0
    for top_class, count in top1[class_][0:6]:
        if top_class not in diff_dict:
            diff_dict[top_class] = []
        if class_ == top_class or (class_, top_class) in already_seen or number_of_pairs >=5:
            continue
        response = generate_attributes(class_, top_class)
        try:
            json_response = process_response(response)
            keys = list(json_response.keys())
            print(keys)
        except:
            print("Could not process the input, continue...")
            continue
            
        diff_dict[class_] += json_response[keys[0]]
        diff_dict[top_class] += json_response[keys[1]]
            
        already_seen.add((class_, top_class))
        already_seen.add((top_class, class_))
        number_of_pairs += 1
        total_pairs += 1
        
        descriptor_json = convert_to_json(diff_dict)
        with open(f'top5_descriptors_cub.json', 'w') as file:
            file.write(descriptor_json)
    

  0%|          | 0/200 [00:00<?, ?it/s]


    Q: What are useful features for distinguishing a Black-footed Albatross from Sooty Albatross in a photo?
    A: There are several useful visual features to tell the difference between a Black-footed Albatross and a Sooty Albatross in a photo
    
{
    "Black-footed Albatross": [
        "has a black back and wings",
        "has a white head and underparts",
        "has a yellow bill with a dark tip",
        "has black feet",
        "has a wingspan of about 6 feet"
    ],
    "Sooty Albatross": [
        "has a dark brown or blackish back and wings",
        "has a white head and underparts",
        "has a pale yellow bill",
        "has pinkish feet",
        "has a wingspan of about 7 feet"
    ]
}
['Black-footed Albatross', 'Sooty Albatross']

    Q: What are useful features for distinguishing a Black-footed Albatross from Laysan Albatross in a photo?
    A: There are several useful visual features to tell the difference between a Black-footed Albatross and a Laysan Albatr

  0%|          | 1/200 [00:19<1:04:11, 19.35s/it]

{
    "Black-footed Albatross": [
        "has a black body with white underparts",
        "has long, narrow wings with a wingspan of up to 7 feet",
        "has a large, hooked bill",
        "has a distinctive black patch around the eyes",
        "has webbed feet for swimming and diving"
    ],
    "Rhinoceros Auklet": [
        "has a black body with a white belly",
        "has shorter wings with a wingspan of about 2 feet",
        "has a short, stout bill with a distinctive horn-like projection on the upper mandible",
        "has a white patch behind the eye",
        "has webbed feet for swimming and diving"
    ]
}
['Black-footed Albatross', 'Rhinoceros Auklet']

    Q: What are useful features for distinguishing a Laysan Albatross from Sooty Albatross in a photo?
    A: There are several useful visual features to tell the difference between a Laysan Albatross and a Sooty Albatross in a photo
    
{
    "Laysan Albatross": [
        "has a white head and neck",
        "has 

  1%|          | 2/200 [00:35<57:00, 17.28s/it]  

...

{
    "Rock Wren": [
        "has a brownish-gray plumage with darker streaks",
        "has a long, slightly curved bill",
        "has a white throat and breast with dark streaks",
        "has a short tail that is often held upright",
        "habitat is rocky areas and desert scrub"
    ],
    "American Pipit": [
        "has a brownish-gray plumage with lighter streaks",
        "has a slender, pointed bill",
        "has a white belly and undertail coverts",
        "has a long tail that is often held horizontally",
        "habitat is open fields and grasslands"
    ]
}
['Rock Wren', 'American Pipit']

    Q: What are useful features for distinguishing a Winter Wren from Rock Wren in a photo?
    A: There are several useful visual features to tell the difference between a Winter Wren and a Rock Wren in a photo
    
{
    "Winter Wren": [
        "has a dark brown color with a mottled pattern",
        "is smaller in size with a compact body",
        "has a short tail that is of

100%|█████████▉| 199/200 [44:52<00:09,  9.16s/it]

{
    "Winter Wren": [
        "has a dark brown color overall",
        "has a short tail that is often held upright",
        "has a thin and pointed bill",
        "has a mottled appearance on the back and wings",
        "has a white throat and belly"
    ],
    "Rusty Blackbird": [
        "has a black color overall",
        "has a longer tail that is often held straight",
        "has a stout and slightly curved bill",
        "has a glossy appearance on the back and wings",
        "has a yellow eye and a yellowish-brown belly"
    ]
}
['Winter Wren', 'Rusty Blackbird']

    Q: What are useful features for distinguishing a Common Yellowthroat from Golden-winged Warbler in a photo?
    A: There are several useful visual features to tell the difference between a Common Yellowthroat and a Golden-winged Warbler in a photo
    
{
    "Common Yellowthroat": [
        "has a yellow throat and breast",
        "has a black mask across its face",
        "has olive-green upperparts",
  

100%|██████████| 200/200 [45:01<00:00, 13.51s/it]

{
    "Common Yellowthroat": [
        "has a yellow throat and breast",
        "has a black mask across its face",
        "has olive-green upperparts",
        "has a white belly",
        "has a relatively small size"
    ],
    "Yellow-bellied Flycatcher": [
        "has a yellow belly",
        "has a pale grayish-green upperparts",
        "has a whitish throat and breast",
        "has a relatively larger size",
        "has a distinct eye-ring"
    ]
}
['Common Yellowthroat', 'Yellow-bellied Flycatcher']


In [32]:
def load_json(filename):
    if not filename.endswith('.json'):
        filename += '.json'
    with open(filename, 'r') as fp:
        return json.load(fp)
    

In [33]:
gpt_descriptions = load_json('top5_descriptors_cub.json')
for key in gpt_descriptions.keys():
    if key not in cub_dataset.classes:
        print(key)

In [34]:
ordDict = OrderedDict()
for class_ in cub_dataset.classes:
    ordDict[class_] = list(set(gpt_descriptions[class_]))

In [35]:
with open(f'descriptors_cub_top5.json', 'w') as file:
    json_string = json.dumps(ordDict, indent=4)
    file.write(json_string)

# Differentiation using the Top-2 Classes

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/16', device=device, jit=False)
model = model.eval()
model = model.requires_grad_(False)

100%|███████████████████████████████████████| 335M/335M [00:38<00:00, 9.04MiB/s]


In [5]:
def compute_encodings(model, labels):
    label_encodings = F.normalize(model.encode_text(clip.tokenize(labels).to(device)))
    return label_encodings

In [6]:
def get_diff_encodings(filename):
    encodings_dict = dict()
    with open(filename, 'r') as fp:
        dict_ = json.load(fp)
        for diff_key in dict_.keys():
            for i, (k, v) in enumerate(dict_[diff_key].items()):
                v.append(f"A photo of a {k} bird")
                v = list(map(lambda x: f"{k} birds {x}", v))
                encoding = F.normalize(model.encode_text(clip.tokenize(v).to(device)))
                if diff_key in encodings_dict:
                    encodings_dict[diff_key][k] = encoding
                else:
                    encodings_dict[diff_key] = {k: encoding}
    return encodings_dict

In [7]:
def label_to_class(class_to_label):
    dataset_items = list(class_to_label.items())
    return dict(map(lambda x: x[::-1], dataset_items))

In [8]:
def aggregate_similarity(similarity_matrix_chunk, aggregation_method='mean'):
    if aggregation_method == 'max': return similarity_matrix_chunk.max()
    elif aggregation_method == 'sum': return similarity_matrix_chunk.sum()
    elif aggregation_method == 'mean': return similarity_matrix_chunk.mean()
    else: raise ValueError("Unknown aggregate_similarity")

In [9]:
#only works for k=2
def find_acc_from_topk(image_encodings, topk_labels, class_to_label, diff_dict):
    pred, topk_diff = [], []
    label_to_class_dict = label_to_class(class_to_label)

    for index, topk_label in enumerate(topk_labels):
        pairs_list, topk_index_to_label = list(), dict()
        
#         len of topk_label is 2

        topk_label1, topk_label2 = int(topk_label[0]), int(topk_label[1])
        class1 = label_to_class_dict[topk_label1]
        class2 = label_to_class_dict[topk_label2]
        key1 = f"{class1} vs {class2}"
        key2 = f"{class2} vs {class1}"
        if key1 in diff_dict:
            pairs_list.append(diff_dict[key1][class1])
            pairs_list.append(diff_dict[key1][class2])
        elif key2 in diff_dict:
            pairs_list.append(diff_dict[key2][class1])
            pairs_list.append(diff_dict[key2][class2])
        else:
            pairs_list.append(compute_encodings(model, class1))
            pairs_list.append(compute_encodings(model, class2))
        
        pairs_sim = [aggregate_similarity(image_encodings[index] @ pairs_list[0].T), 
                      aggregate_similarity(image_encodings[index] @ pairs_list[1].T)]
        
        pairs_sim = torch.stack(pairs_sim)
        pred_label = topk_label[pairs_sim.argmax()]
        pred.append(pred_label)
        
        
    pred = torch.stack(pred)
    return pred

In [10]:
# #for k>2, work in progress
# def find_acc_from_topk(image_encodings, topk_labels, class_to_label, diff_dict):
#     batch_diff = []
#     batch_to_index = []
#     label_to_class_dict = label_to_class(class_to_label)
    
#     for index, topk_label in enumerate(topk_labels):
#         topk_diff, topk_index_to_label = list(), dict()
        
#         for top_index1, top_label1 in enumerate(topk_label[:len(topk_labels)-1]):
#             top_label1 = int(top_label1)
#             topk_index_to_label[top_index1] = top_label1
#             for top_index2, top_label2 in enumerate(topk_label[top_index1+1:]):
#                 top_label2 = int(top_label2)
#                 pairs = []
#                 class1 = label_to_class_dict[top_label1]
#                 class2 = label_to_class_dict[top_label2]
#                 key1 = f"{class1} vs {class2}"
#                 key2 = f"{class2} vs {class1}"
#                 if key1 in diff_dict:
#                     pairs = [diff_dict[key1][class1], diff_dict[key1][class2]]
#                 elif key2 in diff_dict:
#                     pairs = [diff_dict[key2][class1], diff_dict[key2][class2]]
#                 else:
#                     pairs = [compute_encodings(model, class1), compute_encodings(model, class2)]
#                 pair = torch.stack(pairs)
#             topk_index_to_label[len(topk_label) -1] = topk_label[-1]
#         topk_diff = torch.stack(topk_diff)
#         print(f"topk_diff.size(): {topk_diff.size()}")
#         batch_diff.append(topk_diff)
#         batch_to_index.append(topk_index_to_label)
        
#     print(f"len(batch_diff): {len(batch_diff)}")
#     batch_diff = torch.stack(batch_diff)
#     topk_similarity = image_encodings @ batch_diff.T
#     topk_sim_indices = topk_similarity.argmax(dim=1)
#     topk_labels = torch.tensor(map(lambda btoi, index: btoi[index],
#                                    zip(batch_to_index, topk_sim_indices.squeeze())))
#     return topk_labels

In [11]:
def top2_diff_acc(model, dataset, descriptions, diff_loc, batch_size = 64*10, topK = 5):
    
    if topK <= 1:
        print("Top-k should be greater than 1.")
        return ""
    
    diff_dict = get_diff_encodings(diff_loc)
    encodings = compute_encodings(model, descriptions)
    clip_accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(dataset.classes)).to(device)
    clip_accuracy_diff_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(dataset.classes)).to(device)
    clip_accuracy_metric_topk = torchmetrics.Accuracy(top_k=topK, task="multiclass", num_classes=len(dataset.classes)).to(device)
    dataloader = DataLoader(dataset, batch_size, shuffle=True, num_workers=16, pin_memory=True)
    
    for batch_number, batch in enumerate(tqdm(dataloader)):
        images, labels = batch

        images = images.to(device)
        labels = labels.to(device)

        image_encodings = model.encode_image(images)
        image_encodings = F.normalize(image_encodings)

        image_labels_similarity = image_encodings @ encodings.T
        topk_values, topk_labels = image_labels_similarity.topk(topK, dim=1)
        topk_diff_pred = find_acc_from_topk(image_encodings, topk_labels, dataset.class_to_idx ,diff_dict)
        clip_predictions = image_labels_similarity.argmax(dim=1)


        clip_acc = clip_accuracy_metric(image_labels_similarity, labels)
        clip_diff_acc = clip_accuracy_diff_metric(topk_diff_pred, labels)
        clip_acc_topk = clip_accuracy_metric_topk(image_labels_similarity, labels)


    accuracy_logs = {}
    accuracy_logs["Total CLIP-Standard Top-1 Accuracy: "] = 100*clip_accuracy_metric.compute().item()
    accuracy_logs[f"Total CLIP-Standard Top-{topK} Accuracy: "] = 100*clip_accuracy_metric_topk.compute().item()
    accuracy_logs["Total CLIP-diff Accuracy: "] = 100*clip_accuracy_diff_metric.compute().item()


    
    return accuracy_logs

In [12]:
res = top2_diff_acc(model, cub_dataset, cub_dataset.classes, 'descriptors/descriptors_cub_diff.json',
         topK = 2)

for key, val in res.items():
    print(key, val)

AttributeError: 'list' object has no attribute 'items'

# CUB

# Oxford Pets